In [1]:
import tensorflow as tf
import utils

In [2]:
df_train, df_val, df_test = utils.load_data()
print(f"df_train.shape = {df_train.shape}, df_val.shape = {df_val.shape}, df_test.shape = {df_test.shape}")

df_train.shape = (49063, 19), df_val.shape = (14018, 19), df_test.shape = (7010, 19)


In [3]:
df_train.head(10)

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),Wx,Wy,max Wx,max Wy,Day sin,Day cos,Year sin,Year cos
5,0.945308,-1.982473,-2.041888,-1.918973,1.117102,-1.302851,-1.477323,-0.790424,-1.480036,-1.482697,2.218524,0.193409,0.221161,0.111140,0.217928,-1.366064,-0.366026,-0.069373,1.427981
11,0.959770,-2.078372,-2.138166,-2.060964,1.044617,-1.330143,-1.534354,-0.786272,-1.536190,-1.539035,2.325708,0.172987,0.222101,0.109458,0.227798,-1.414249,0.000015,-0.068349,1.427977
17,0.986284,-2.070284,-2.132435,-2.045187,1.062738,-1.328843,-1.527225,-0.788348,-1.528703,-1.531992,2.323998,0.207983,0.276266,0.111218,0.324078,-1.366064,0.366057,-0.067326,1.427971
23,1.004362,-2.098014,-2.161090,-2.096820,1.008375,-1.336641,-1.546235,-0.782121,-1.547420,-1.553119,2.358913,0.270343,0.195267,0.246907,0.145176,-1.224793,0.707153,-0.066303,1.427965
29,1.061006,-2.165028,-2.232152,-2.187178,0.984214,-1.353535,-1.579503,-0.782121,-1.581113,-1.585982,2.446320,0.112264,0.350818,0.048640,0.402053,-1.000062,1.000060,-0.065280,1.427958
35,1.091135,-2.169649,-2.239029,-2.182875,1.014416,-1.354835,-1.579503,-0.786272,-1.581113,-1.583635,2.459016,0.274590,0.268633,0.175115,0.377528,-0.707188,1.224815,-0.064257,1.427951
41,1.165857,-2.111879,-2.186305,-2.108294,1.026496,-1.340539,-1.550988,-0.784197,-1.554908,-1.557814,2.416777,1.308822,-0.008002,1.211039,0.057261,-0.366129,1.366101,-0.063234,1.427942
47,1.267093,-1.988250,-2.071689,-1.957698,1.020456,-1.305450,-1.493957,-0.780045,-1.495011,-1.499129,2.311058,-0.001703,-0.006445,-0.024096,0.004664,-0.000127,1.414292,-0.062211,1.427933
53,1.329762,-1.937412,-2.025842,-1.927578,0.923810,-1.289855,-1.479699,-0.767591,-1.483780,-1.487392,2.274191,0.138795,0.168457,0.121145,0.179932,0.365875,1.366101,-0.061188,1.427924
59,1.399663,-1.865777,-1.959365,-1.831483,0.947972,-1.267762,-1.439303,-0.767591,-1.446344,-1.449834,2.216082,0.572007,-0.606556,0.676312,-0.786448,0.706934,1.224815,-0.060165,1.427913


In [4]:
window_size = 6
shuffle_buffer = 1000
batch_size =32

In [5]:
ds = tf.data.Dataset.from_tensor_slices(df_train)
ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda window: window.batch(window_size + 1))
ds = ds.map(lambda window: (window[:-1], window[-1:]))
ds = ds.cache()
# ds = ds.shuffle(shuffle_buffer)
ds = ds.batch(batch_size).prefetch(1)

In [6]:
for x_batch, y_batch in ds.take(1):
    print(f"x_batch.shape = {x_batch.shape}")
    print(f"y_batch.shape = {y_batch.shape}")
    break

x_batch.shape = (32, 6, 19)
y_batch.shape = (32, 1, 19)


In [7]:
x_batch[0]

<tf.Tensor: shape=(6, 19), dtype=float64, numpy=
array([[ 9.45307599e-01, -1.98247323e+00, -2.04188844e+00,
        -1.91897277e+00,  1.11710152e+00, -1.30285119e+00,
        -1.47732321e+00, -7.90423621e-01, -1.48003637e+00,
        -1.48269721e+00,  2.21852381e+00,  1.93409239e-01,
         2.21161294e-01,  1.11140455e-01,  2.17927873e-01,
        -1.36606404e+00, -3.66026353e-01, -6.93725796e-02,
         1.42798143e+00],
       [ 9.59769847e-01, -2.07837211e+00, -2.13816632e+00,
        -2.06096373e+00,  1.04461734e+00, -1.33014257e+00,
        -1.53435391e+00, -7.86272298e-01, -1.53618981e+00,
        -1.53903452e+00,  2.32570755e+00,  1.72986774e-01,
         2.22100866e-01,  1.09458245e-01,  2.27798499e-01,
        -1.41424908e+00,  1.52973595e-05, -6.83494637e-02,
         1.42797673e+00],
       [ 9.86283967e-01, -2.07028426e+00, -2.13243549e+00,
        -2.04518696e+00,  1.06273839e+00, -1.32884298e+00,
        -1.52722507e+00, -7.88347960e-01, -1.52870268e+00,
        -1.531

In [8]:
y_batch[0]

<tf.Tensor: shape=(1, 19), dtype=float64, numpy=
array([[ 1.16585687, -2.11187895, -2.18630526, -2.10829405,  1.0264963 ,
        -1.34053929, -1.55098787, -0.78419664, -1.55490762, -1.55781362,
         2.41677711,  1.30882225, -0.00800171,  1.2110392 ,  0.05726148,
        -0.36612879,  1.36610133, -0.06323395,  1.42794235]])>